# LSTM Modelling

On this Notebook, a LSTM Model will be tried for the Competitition Data.

## Libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
sns.set_style('darkgrid')
from cnr_methods import get_selected_features, transform_data, revert_data,metric_cnr, get_simplified_data
import random

from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

## Read Data

Here, the data used correspond to the results of the Feature Engineering and Selection Step. For simplicity, during Hyperparameter Optimization, only Wind Farm 3 Training Data is used.

In [2]:
full_data = get_selected_features(0)

#full_data = full_data.rename({'Unnamed: 0' : 'Time'},axis=1)
full_data = full_data.set_index('Time')

full_label = pd.read_csv('Data/Y_train.csv')
X = full_data[full_data['Set']=='Train']

WF = 'WF3'
X = X[X['WF']==WF]
y = full_label[full_label['ID'].isin(X['ID'])]

In [3]:
X = X.drop(['ID','WF','Set'],axis=1)

In [4]:
X.head()

,U_100m,V_100m,U_10m,V_10m,T,CLCT,Wind Direction 100m,Wind Direction 10m,Wind Speed 100m,V_100m_lag_21_days,...,V_10m_Rolling_14_Window_Variance,U_100m_Rolling_14_Window_Mean,Wind Speed 10m,V_100m_Rolling_7_Window_Mean,T_lag_21_days,U_10m_Rolling_14_Window_Variance,CLCT_Rolling_7_Window_Mean,V_100m_lag_7_days,U_10m_lag_7_days,U_100m_lag_21_days
Time,,,,,,,,,,,,,,,,,,,,,
2018-05-01 01:00:00,5.789500,3.820200,1.054669,1.317597,275.690,86.504507,0.583268,0.895782,6.936299,NaN,...,NaN,NaN,1.687717,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 02:00:00,6.003300,3.920600,0.876879,1.483483,275.770,98.976088,0.578533,1.036951,7.170127,NaN,...,NaN,NaN,1.723264,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 03:00:00,5.931829,0.907656,0.949640,1.419591,276.875,64.193607,0.151837,0.981212,6.000870,NaN,...,NaN,NaN,1.707938,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 04:00:00,5.205300,1.683800,1.027462,1.029786,275.650,57.482484,0.312855,0.786528,5.470862,NaN,...,NaN,NaN,1.454695,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 05:00:00,4.845900,0.702200,1.011645,0.785352,275.530,89.971463,0.143904,0.660129,4.896512,NaN,...,NaN,NaN,1.280704,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fill NaN's

In [5]:
X = X.fillna(method="ffill", axis=1) # ZOH
X = X.fillna(0)

In [6]:
y = y.fillna(method="ffill", axis=1) # ZOH
y = y.fillna(0)

## Scaling Data

In [7]:
directions = X[['Wind Direction 100m', 'Wind Direction 10m']]
directions["Sin_Wind Direction 100m"] = np.sin(X['Wind Direction 100m']*(np.pi/180))
directions["Cos_Wind Direction 100m"] = np.cos(X['Wind Direction 100m']*(np.pi/180))
directions["Sin_Wind Direction 10m"] = np.sin(X['Wind Direction 10m']*(np.pi/180))
directions["Cos_Wind Direction 10m"] = np.cos(X['Wind Direction 10m']*(np.pi/180))

directions = directions.drop(['Wind Direction 100m', 'Wind Direction 10m'],axis=1)
directions = directions.reset_index().drop('Time',axis=1)

In [8]:
X = X.drop(['Wind Direction 100m', 'Wind Direction 10m'],axis=1)
X_columns = X.columns

In [9]:
scaler = MinMaxScaler(feature_range=(-1,1))
X = scaler.fit_transform(X)
X = pd.DataFrame(X,columns=X_columns)

In [10]:
X = pd.concat([X,directions],axis=1)

In [11]:
#y = scaler.fit_transform(y)
#y = pd.DataFrame(y)

In [12]:
X.head()

,U_100m,V_100m,U_10m,V_10m,T,CLCT,Wind Speed 100m,V_100m_lag_21_days,U_100m_Rolling_14_Window_Variance,U_10m_Rolling_7_Window_Mean,...,T_lag_21_days,U_10m_Rolling_14_Window_Variance,CLCT_Rolling_7_Window_Mean,V_100m_lag_7_days,U_10m_lag_7_days,U_100m_lag_21_days,Sin_Wind Direction 100m,Cos_Wind Direction 100m,Sin_Wind Direction 10m,Cos_Wind Direction 10m
0,0.228968,0.130650,-0.080167,0.046187,-0.652228,0.730090,-0.235387,0.324928,-0.660959,0.667455,...,-0.990458,-0.724366,-0.966246,-0.664663,-0.797110,-0.701557,0.010180,0.999948,0.015634,0.999878
1,0.243537,0.136909,-0.099786,0.062691,-0.648044,0.979522,-0.209145,0.339507,-0.649466,0.695576,...,-0.990227,-0.718540,-0.965535,-0.663918,-0.796306,-0.700795,0.010097,0.999949,0.018097,0.999836
2,0.238667,-0.050938,-0.091757,0.056334,-0.590239,0.283872,-0.340364,0.266607,-0.706936,0.554956,...,-0.990327,-0.721052,-0.965841,-0.664239,-0.796652,-0.701124,0.002650,0.999996,0.017125,0.999853
3,0.189158,-0.002548,-0.083169,0.017554,-0.654321,0.149650,-0.399844,0.233563,-0.732986,0.491215,...,-0.991976,-0.762557,-0.970906,-0.669546,-0.802382,-0.706548,0.005460,0.999985,0.013727,0.999906
4,0.164667,-0.063747,-0.084914,-0.006764,-0.660598,0.799429,-0.464300,0.197754,-0.761215,0.422142,...,-0.993110,-0.791074,-0.974386,-0.673192,-0.806318,-0.710275,0.002512,0.999997,0.011521,0.999934


In [13]:
y

,ID,Production
12478,12479.0,1.49
12479,12480.0,2.48
12480,12481.0,2.21
12481,12482.0,1.49
12482,12483.0,1.25
...,...,...
18712,18713.0,1.56
18713,18714.0,1.29
18714,18715.0,0.89
18715,18716.0,1.08


## Subsets Creation

Convert Df's to Array on Correct Shape

In [14]:
n_steps = 7

In [15]:
# split a multivariate sequence into samples
def split_sequences(X, y, n_steps):
	sample_X, sample_y = list(), list()
	for i in range(len(X)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(X):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = X[i:end_ix, :-1], y[end_ix-1,-1]
		sample_X.append(seq_x)
		sample_y.append(seq_y)
	return np.array(sample_X), np.array(sample_y)

In [16]:
sample_X,sample_y = split_sequences(X.values,y.values,n_steps)

In [17]:
n_features = sample_X.shape[2]

## Model

Model Creation Function:

In [18]:
def LSTM_Model(input_shape, batch_size=1):
  # Numerical branch

  input_layer = tf.keras.Input(shape = input_shape,batch_size = batch_size)

  hidden_1 = tf.keras.layers.LSTM(units=1024,return_sequences=True,stateful=True)(input_layer)
  hidden_1 = tf.keras.layers.Dropout(0.3)(hidden_1)
  hidden_1 = tf.keras.layers.BatchNormalization()(hidden_1)

  hidden_2 = tf.keras.layers.LSTM(units=512, stateful=True)(hidden_1)
  hidden_2 = tf.keras.layers.Dropout(0.3)(hidden_2)
  hidden_2 = tf.keras.layers.BatchNormalization()(hidden_2)


  # Output
  outputs = tf.keras.layers.PReLU()(hidden_2)
  outputs = tf.keras.layers.Dropout(rate=0.3)(outputs)
  outputs = tf.keras.layers.BatchNormalization()(outputs)
  outputs = tf.keras.layers.Dense(units=1)(outputs)

  model = tf.keras.Model(inputs=input_layer, outputs=outputs)

  return model

In [19]:
input_shape = (n_steps,n_features)

In [20]:
model = LSTM_Model(input_shape)

In [21]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(1, 7, 94)]              0         
_________________________________________________________________
lstm (LSTM)                  (1, 7, 1024)              4583424   
_________________________________________________________________
dropout (Dropout)            (1, 7, 1024)              0         
_________________________________________________________________
batch_normalization (BatchNo (1, 7, 1024)              4096      
_________________________________________________________________
lstm_1 (LSTM)                (1, 512)                  3147776   
_________________________________________________________________
dropout_1 (Dropout)          (1, 512)                  0         
_________________________________________________________________
batch_normalization_1 (Batch (1, 512)                  2048  

## Validation

In [22]:
def metric_cnr(preds,labels):
    cape_cnr = 100*tf.math.reduce_sum(tf.math.abs(preds-labels))/tf.math.reduce_sum(labels)
    return cape_cnr

In [23]:
random.seed(317)
tf.random.set_seed(317)

patience = 5
epochs = 25
k_fold_splits = 5
total_it = 100
monitor = "root_mean_squared_error"
#batch_size = 1 * ((len(X) - len(X) // k_fold_splits) // (total_it))
batch_size = 1

In [24]:
# Define Time Split Cross Validation
tscv = TimeSeriesSplit(n_splits=k_fold_splits)

# Separating Data from Hold Out Set

X_cv, _, y_cv, _ = train_test_split(X, y, test_size=0.125, shuffle=False)

train_scores = np.empty(0)
val_scores = np.empty(0)
test_scores = np.empty(0)
for train_index, test_index in tscv.split(X_cv):

    # Get the Data of the Split
    X_train, X_test = X_cv.iloc[train_index], X_cv.iloc[test_index]
    y_train, y_test = y_cv.iloc[train_index], y_cv.iloc[test_index]

    # Separating Training Set of Split on Train and Validation Subsets
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.143, shuffle=False)

    # Reshape Data
    X_train, y_train = split_sequences(X_train.values,y_train.values,n_steps)
    X_val, y_val = split_sequences(X_val.values,y_val.values,n_steps)
    X_test, y_test = split_sequences(X_test.values,y_test.values,n_steps)

    # Create Model
    model = LSTM_Model(input_shape,batch_size=batch_size)

    # Callbacks
    callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=patience, verbose=0, min_delta=1e-8)]

    # Train the Model
    model.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError(),metric_cnr])
    history = model.fit(x = X_train, y = y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, y_val), callbacks=callbacks_list,shuffle=False)

    # Train and Validation Score
    train_score = np.array(history.history['metric_cnr']).mean()
    val_score = np.array(history.history['val_metric_cnr']).mean()

    # Test Score
    preds = model.predict(X_test,batch_size = batch_size,callbacks=callbacks_list)
    test_score = metric_cnr(preds,y_test)

    train_scores = np.append(train_scores,train_score)
    val_scores = np.append(val_scores,val_score)
    test_scores = np.append(test_scores,test_score)

Epoch 1/25
777/777 [==============================] - 105s 135ms/step - loss: 3.8686 - root_mean_squared_error: 1.9669 - metric_cnr: inf - val_loss: 25552.0195 - val_root_mean_squared_error: 159.8500 - val_metric_cnr: -81.6220
Epoch 2/25
777/777 [==============================] - 105s 136ms/step - loss: 3.8372 - root_mean_squared_error: 1.9589 - metric_cnr: 92.2569 - val_loss: 44466.8789 - val_root_mean_squared_error: 210.8717 - val_metric_cnr: -81.8409
Epoch 3/25
777/777 [==============================] - 104s 134ms/step - loss: 3.8249 - root_mean_squared_error: 1.9557 - metric_cnr: 91.0197 - val_loss: 40149.9922 - val_root_mean_squared_error: 200.3746 - val_metric_cnr: -83.4299
Epoch 4/25
777/777 [==============================] - 104s 134ms/step - loss: 3.8192 - root_mean_squared_error: 1.9543 - metric_cnr: 90.0198 - val_loss: 32507.7383 - val_root_mean_squared_error: 180.2990 - val_metric_cnr: -85.2031
Epoch 5/25
777/777 [==============================] - 105s 135ms/step - loss: 3.

KeyboardInterrupt: 

In [25]:
last_train_index = []
for train_index, test_index in tscv.split(X):
    print(type(train_index))
    last_train_index = train_index

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


### Hold Out Score

In [26]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.125, shuffle=False)

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.143, shuffle=False)

In [28]:
X_train, y_train = split_sequences(X_train.values,y_train.values,n_steps)
X_val, y_val = split_sequences(X_val.values,y_val.values,n_steps)
X_holdout, y_holdout = split_sequences(X_holdout.values,y_holdout.values,n_steps)

In [29]:
model = LSTM_Model(input_shape)

In [30]:
callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor=monitor, patience=patience, verbose=0, min_delta=1e-8)]
model.compile(optimizer='adam', loss='hinge', metrics=[tf.keras.metrics.RootMeanSquaredError(),metric_cnr])
model.fit(x = X_train, y = y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_val, y_val), callbacks=callbacks_list)

Epoch 1/25


InvalidArgumentError:  [_Derived_]  Specified a list with shape [1,94] from a tensor with shape [367,94]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[model_2/lstm_4/StatefulPartitionedCall]] [Op:__inference_train_function_13094]

Function call stack:
train_function -> train_function -> train_function


In [31]:
preds = model.predict(X_holdout,batch_size = batch_size,callbacks=callbacks_list)

InvalidArgumentError:  [_Derived_]  Specified a list with shape [1,94] from a tensor with shape [367,94]
	 [[{{node TensorArrayUnstack/TensorListFromTensor}}]]
	 [[model_2/lstm_4/StatefulPartitionedCall]] [Op:__inference_predict_function_14081]

Function call stack:
predict_function -> predict_function -> predict_function


In [32]:
plt.figure(figsize=(15,8))
preds_len = np.arange(len(y_holdout))
plt.plot(preds_len,y_holdout,label='True Values')
plt.plot(preds_len,preds,'r--',label='Predicts')
plt.legend()

NameError: name 'preds' is not defined